In [ ]:
!pip install urllib3

In [2]:
# origs
CAR_IMAGE_LINK = "https://proproprogs.ru/htm/nn_pytorch/files/pytorch-svertochnye-neyronnye-seti-vgg-16-i-vgg-19.files/image003.jpg"
PATRICK_BATEMAN_LINK = "https://avatars.mds.yandex.net/i?id=65a7105a5e792e453a3297ed7efa063f_l-12384085-images-thumbs&n=13"
#styles
PAINTING1_IMAGE_LINK = "https://i.pinimg.com/736x/f4/fe/95/f4fe9569c69037636b41ada357efe559.jpg"
PAINTING2_IMAGE_LINK = "https://avatars.mds.yandex.net/i?id=561355cb5cb6fed9acdcc60ccc12358c_l-5505574-images-thumbs&n=13"
PAINTING3_IMAGE_LINK = "https://raw.githubusercontent.com/kirill-ionkin/NST_Gatys/refs/heads/main/example/style_1.jpg"
GRAFFITI1_IMAGE_LINK = "https://yandex-images.clstorage.net/qtS9G7366/c733613yWfJ/HO0gQjXc1ZbGC1QvWLbP1GxS7oziTDPCbb0JJFoAeLU_MBlsevzXrFnczv8eyTDeJYTZ4E0hMGRQTtt_YInymUq99RPl3BSUgpkTK8hyzJc8kesYchkwVONllOyTzvMtILkTculplTsYW0k7zoepDzbVxGwYaOA4hJBWMiEYrorX4azFvY4TEgKdlmIas8QXMhApinHMoAHiK9EtkMt84mv7nf9vrJb431TXsoEG71p6Hc7iG6Kp887fETgkVO_AVSkrivSF3VwDWldlEbUInn1T6oi0CuEAbq6fZhMDsicuNN-woyOcOlVdGvMIRiVTttMMKRDo6LwNGpDw4NOqyp2uIMP7ixLTTJlSo5k6xRr1mqMH-sPhRqDjkahajTzsYLRV8-pilTDS1Za2y41nEjqaTONabGG8hhQaPaWW6sacKuPHtA7aWcEbkuoatorb8ZHlSHuCqYRhZ1TgUAlyb-s2UHDpJRu-15keMwcIp9m6XMuglepi-InS2XcgFy6NFajizHoPlVqH0RQhGrUMHDXb4Uz8Qu5DLSdfKNPKsuWuutpwK2eauNnTETkOzuXRdliD4petrrMH0x07Llhvx1ckaIo7z9dSypRbJ9u8RF97mSwBsA8pTOhrVGAfwXas4nnbtKGtE3aTU1a_AwblGzXZROaXqqj1TFoZ-yCe5YUYraNMsUuf34eV2aFX9MwfNlcniHhHIoaqo9FsXUwxLCdzU3MpbF-w3Z5TfAZFJB3xHE3lV2rhu4cQET5snqvHHyjsRzzBE99Hk5EiFrtC2PPcZUm_zOjNYSDd4pDFuyBre5C_4ajTNlsb2fwAwaTbsh0MbBpn6HeJ2dq_ZhpjCd5sIEV7glVUwhMS6Vh8z5KzEeVNsshhSKzsVuGbDzpgLrWfeaqln3KR3FO-AwFoVD3Xzu6aLaZ0zFOYsGkeZAzS4aJENMlQkoAWW6ITtsTbe5GjyHhOoQjgqZ4sUA"
GRAFFITI2_IMAGE_LINK = "https://avatars.mds.yandex.net/i?id=a9c7baaf914d26034afa9808f0729dc9_l-5365143-images-thumbs&n=13"

In [3]:
from PIL import Image
import requests

def image_by_link(url: str):
  return Image.open(requests.get(url, stream=True).raw)

img_orig = image_by_link(PATRICK_BATEMAN_LINK).convert("RGB")
img_style = image_by_link(PAINTING3_IMAGE_LINK).convert("RGB")

In [ ]:
display(img_orig)
display(img_style)

In [ ]:
import torch

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [6]:

import torchvision.models as models
import torch.nn as nn

class StyleModel(nn.Module):
  def __init__(self) -> None:
    super().__init__()
    self._model = models.vgg19(weights=models.VGG19_Weights.DEFAULT)
    self.transforms = models.VGG19_Weights.IMAGENET1K_V1.transforms()
    self.requires_grad_(False)
    self._model.features.requires_grad_(False)
    self._model.features.eval()
    self.layers = self._model.features
    self.idx = (0, 5, 10, 19, 28, 34)

  def forward(self, x):
    outputs = []
    for i, layer in enumerate(self.layers):
      x = layer(x)
      if i in self.idx:
        outputs.append(x.squeeze())

    return outputs[:-1], outputs[-1]

In [ ]:
import torch.optim as optim
import torchvision.transforms.v2 as tfs_v2
from tqdm import tqdm

def loss_content_eval(orig, x):
  return ((orig - x).square()).mean()

def gram_matrix(x):
  gram = []
  for i in range(len(x)):
    a = x[i].squeeze().view(x[i].shape[0], -1)
    gram.append(a.mm(a.mT) / a.shape[1])
  return  gram

def loss_style_eval(gram_style, gram, weights=[1., 0.8, 0.5, 0.3, 0.1]):
  loss = 0
  for i in range(len(gram_style)):
    loss += ((gram_style[i] - gram[i]) ** 2).mean() * weights[i]
  return loss

def normalize(x):
  return (x - x.min()) / (x.max() - x.min())

# create model
model = StyleModel().to(device)
model.eval()

# transform images
transforms = tfs_v2.Compose([
  tfs_v2.ToImage(),
  tfs_v2.ToDtype(torch.float32, scale=True),
])

img_orig = transforms(img_orig).to(device)
img_style = transforms(img_style).to(device)

In [ ]:
# create image we're stylizing
img = img_orig.clone()
img.requires_grad_(True)

# evaluate content and style tensors
style_orig, content_orig = model(img)
style_style, _ = model(img_style)

gram_style = gram_matrix(style_style)

# detach constants
content_orig = content_orig.detach()
gram_style = [i.detach() for i in gram_style]

# hyperparameters
epochs = 2000
opt = optim.Adam(params=[img], lr=0.01)
content_coeff = 1
style_coeff = 1e6

# training loop
progress_bar = tqdm(range(epochs), desc="Style Transfer Progress")
for _ in progress_bar:
  style, content = model(img)
  gram = gram_matrix(style)

  loss_content = loss_content_eval(content, content_orig)
  loss_style = loss_style_eval(gram_style, gram)
  loss = content_coeff * loss_content + style_coeff * loss_style
  progress_bar.set_postfix(loss_content=loss_content.item(), loss_style=loss_style.item(), loss=loss.item())

  opt.zero_grad()
  loss.backward()
  opt.step()

  img.data.clamp_(0, 1)

In [ ]:
x = img.requires_grad_(False)
x = x.clamp(0, 1) * 255.0
x = x.permute(1, 2, 0)
x = x.cpu().numpy()
x = x.astype('uint8')
Image.fromarray(x, "RGB")